Xception

In [1]:
#import modules
import numpy as np  #linear algebra
import pandas as pd   #data processing, CSV @ excel file I/O (e.g. pd.read_csv)
import tensorflow as tf
import cv2
from glob import glob
import matplotlib.pyplot as plt
import seaborn as sns

Path for normal, covid-19, TB
Grab all image files for every class

In [2]:
#define paths
normalpath = 'D:\\dataset300\\normal'
covidpath = 'D:\\dataset300\\covid'
tbpath = 'D:\\dataset300\\tb'

# Use glob to grab images from path .jpg or jpeg
normal_files = glob(normalpath + '/*')
covid_files = glob(covidpath + '/*')
tb_files = glob(tbpath + '/*')

In [3]:
#Store all images into a variable
allfiles = normal_files + covid_files + tb_files

In [ ]:
print('There are %d normal cases.' % len(normal_files))
print('There are %d covid positive cases.' % len(covid_files))
print('There are %d tuberculosis positive cases.' % len(tb_files))

In [ ]:
#show xray normal
f = plt.figure(figsize=(15,7))
f.suptitle('NORMAL',fontsize=20)
f.subplots_adjust(top=2.35)
for i in range(3):
    sp = f.add_subplot(1,3,i+1)
    img = cv2.imread(normal_files[i])
    img_gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    plt.tick_params(labelbottom=False,
                labelleft=False,
                labelright=False,
                labeltop=False)
    plt.imshow(img_gray)
    plt.gray()
plt.show()

In [ ]:
#show xray covid
f = plt.figure(figsize=(15,7))
f.suptitle('COVID19',fontsize=20)
f.subplots_adjust(top=2.35)
for i in range(3):
    sp = f.add_subplot(1,3,i+1)
    img = cv2.imread(covid_files[i])
    img_gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    plt.tick_params(labelbottom=False,
                labelleft=False,
                labelright=False,
                labeltop=False)
    plt.imshow(img_gray)
    plt.gray()
plt.show()

In [ ]:
#show xray tb
f = plt.figure(figsize=(15,7))
f.suptitle('TB',fontsize=20)
f.subplots_adjust(top=2.35)
for i in range(3):
    sp = f.add_subplot(1,3,i+1)
    img = cv2.imread(tb_files[i])
    img_gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    plt.tick_params(labelbottom=False,
                labelleft=False,
                labelright=False,
                labeltop=False)
    plt.imshow(img_gray)
    plt.gray()
plt.show()

Preparing labels (0=NORMAL, 1=COVID19, 2=TB)

In [4]:
# Preparing Labels

import cv2

labels = []


for i in range(len(normal_files)):
  labels.append(0) #atau mungkin kena letak num like 0

for i in range(len(covid_files)):
  labels.append(1)

for i in range(len(tb_files)):
  labels.append(2)

List of images for all classes

In [5]:
images = []
for i in range(len(normal_files)):
    normfile = normal_files[i]
    images.append(normfile)
for i in range(len(covid_files)):
    covfile = covid_files[i]
    images.append(covid_files)
for i in range(len(tb_files)):
    tbfile = tb_files[i]
    images.append(tb_files)

Create dataframe containing file name with labels of Xray_class

In [6]:
img_df = pd.DataFrame({'Image_name':allfiles, 'Xray_class': labels})

In [7]:
import numpy as np
from keras.applications import Xception
from keras.preprocessing import image
from keras.models import Model
from keras.applications.xception import preprocess_input


Create model Xception

In [8]:
base_model = Xception(weights="imagenet")

In [9]:
model = Model(inputs=base_model.input, outputs=base_model.get_layer('avg_pool').output)

In [10]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 299, 299, 3  0           []                               
                                )]                                                                
                                                                                                  
 block1_conv1 (Conv2D)          (None, 149, 149, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 block1_conv1_bn (BatchNormaliz  (None, 149, 149, 32  128        ['block1_conv1[0][0]']           
 ation)                         )                                                             

Use pretrained Xception for feature extraction

In [11]:
from keras.utils import load_img
from keras.utils import img_to_array

In [12]:
# Output of the layer before the final classification layer: shape (1, 2048)

image_size = 299
features_array = np.zeros((900,2048))

for i, img_path in enumerate(allfiles):
    img = load_img(img_path, target_size=(image_size, image_size))
    x = img_to_array(img)
    x = np.expand_dims(x, axis=0)
    X = preprocess_input(x)
    features = model.predict(X)
    features = features.reshape(2048,)
    features_array[i,:] = features

1/1 [==============================] - 0s 401ms/step


In [13]:
features_array.shape

(900, 2048)

In [ ]:
X.shape

In [ ]:
img_df.head()

In [14]:
df = pd.DataFrame(features_array)
df['Image_name'] = img_df.Image_name
df['Xray_class'] = img_df.Xray_class
df.head()

,0,1,2,3,4,5,6,7,8,9,...,2040,2041,2042,2043,2044,2045,2046,2047,Image_name,Xray_class
0,0.025444,0.650957,0.121822,0.225538,0.016219,0.022862,0.000000,0.532824,0.027664,0.009648,...,0.163416,0.151689,0.231440,0.000000,0.346852,0.000000,0.543420,0.009617,D:\dataset300\normal\Normal-315.png,0
1,0.067924,0.177339,0.020034,0.000000,0.061072,0.040595,0.023707,0.406153,0.006811,0.002479,...,0.151441,0.084369,0.096323,0.150834,0.047081,0.000000,0.000911,0.053235,D:\dataset300\normal\Normal-2.png,0
2,0.196098,0.215693,0.011934,0.028829,0.118768,0.025659,0.087629,0.124655,0.100190,0.101006,...,0.196131,0.099611,0.263022,0.376871,0.061850,0.000185,0.001383,0.108453,D:\dataset300\normal\Normal-3.png,0
3,0.000374,0.385093,0.093754,0.014193,0.031064,0.011863,0.029341,0.000000,0.011486,0.067398,...,0.100627,0.169006,0.097124,0.631717,0.000000,0.001187,0.000126,0.051582,D:\dataset300\normal\Normal-4.png,0
4,0.203158,0.420689,0.047584,0.021112,0.047137,0.080540,0.165093,0.018547,0.192010,0.365929,...,0.049643,0.285309,0.055261,0.268497,0.114053,0.000000,0.102443,0.000000,D:\dataset300\normal\Normal-5.png,0


In [ ]:
#save extracted features into EXCEL FILE
df.to_excel("features_Xception.xlsx")  